# Modeling joint distributions using Turing

The goal of Bayesian data analysis is to come up with a plausible explanation for some phenomenon or data in the form of a joint distribution. Defining complicated multivariate distributions outright is extremely difficult, so statistician typically decompose the task into defining conditional distributions and place independence assumptions on the interactions between the random variables.

To get a proper introduction to Turing.jl, its syntax and everything surrounding the ecosystem, see [https://turing.ml/](https://turing.ml/stable/).

# Model syntax

Turing syntax is designed to closely resemble the typical mathematical notation for Bayesian models.

## Model definition

### `@model`

A Turing model is defined by using the model macro as follows:

In [1]:
using Turing

@model function demo(y; σ²=1.0)
    n=length(y)

    σ² ~ InverseGamma(2, 3)
    σ = √σ²
    x ~ Normal(0, σ)

    y ~ MvNormal(x * ones(n), 1.0)
end

demo (generic function with 1 method)

where the right-hand side of `~` is expected to be a `Distribution` from [Distributions.jl](https://github.com/JuliaStats/Distributions.jl).

Notice that there's seemingly nothing special about the body of the function above, with the exception of the `~` statements! In fact, _any_ Julia code can go into a Turing model, e.g. [solving differential equations](https://turing.ml/dev/tutorials/10-bayesian-differential-equations/)! 

All that `@model` does is to replace the `~` statements with some function calls and it adds some additional arguments to the function that Turing uses internally to keep track of everything. If you're very keen on understanding some of its internals, check out [https://turing.ml/dev/docs/for-developers/compiler](https://turing.ml/dev/docs/for-developers/compiler).

A Turing model is then instantiated with some data, e.g. `demo(randn(10))`, to condition the model on said data:

In [2]:
m = demo(ones(10));

To sample from the `Model` we simply call it:

In [3]:
m()

10-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

Note how the observations was returned back to us for the model call. This is not because Turing has some feature that it always returns the observations or anything, no no no, it's simply that `y ~ ...` was the last statement in the model, hence it was returned. This is exactly the same behavior as a standard Julia function! Often one might want to make the return value something different, e.g.

In [4]:
@model function demo(y; σ²=1.0)
    n = length(y)

    σ² ~ InverseGamma(2, 3)
    σ = √σ²
    x ~ Normal(0, σ)

    y ~ MvNormal(x * ones(n), 1.0)

    return (; σ, x, y, whatever=100 .* y)
end

m = demo(ones(10));
m()

(σ = 1.0, x = 0.7551512526930272, y = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], whatever = [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0])

This can be extremely useful for debugging but also if there are certain quantities one might want to compute from the model. As we'll see later, there's a `generated_quantities` function that allows you to get the return-values of the model _conditioned_ on posterior samples.

### Usage of `missing`: sampling observations

We can also instantiate `demo` _without_ any observations by simply setting `y` to `missing`, and in Turing a `missing` variable is considered as a random variable to be sampled rather than an observation. 

But note when we do this for the above `demo` model, the default `n = length(y)` doesn't make sense anymore since `length(::Missing)` isn't defined, i.e. we need to specify the length of the variable `y` when it's `missing`:

In [5]:
# If `y` is not an array, we default to using `n=1`.
@model function demo(y; σ²=1.0, n = y isa AbstractArray ? length(y) : 1)
    σ² ~ InverseGamma(2, 3)
    σ = √σ²
    x ~ Normal(0, σ)

    y ~ MvNormal(x * ones(n), 1.0)

    return (; σ, x, y, whatever=100 .* y)
end

m = demo(missing, n=10)
m()

(σ = 1.0, x = -0.9470887353923412, y = [-0.4966704092922676, -0.9802262472224543, -2.0684170042881345, -1.308281109013812, -0.38466325572132387, -0.16784824343988825, -2.6450263246008276, 0.6694487817739021, -0.7887400747487461, -2.9013218108240055], whatever = [-49.66704092922676, -98.02262472224544, -206.84170042881345, -130.8281109013812, -38.46632557213239, -16.784824343988824, -264.50263246008274, 66.94487817739021, -78.8740074748746, -290.13218108240056])

We can do the same for the keyword argument `σ²`:

In [6]:
m = demo(missing; σ² = missing, n=10)
m()

(σ = 1.1503460317662495, x = 1.402065757613783, y = [1.0173912416143962, 1.645529225907439, 0.7607617591417798, 0.007716147223924219, -1.1481330756305723, 2.5297646933134494, 2.6536747465136488, -0.49334734068235697, 1.4875121678687626, 1.8834481053204142], whatever = [101.73912416143962, 164.5529225907439, 76.07617591417798, 0.7716147223924219, -114.81330756305724, 252.97646933134496, 265.3674746513649, -49.3347340682357, 148.75121678687626, 188.3448105320414])

## Loops

To model a random vector, where the components depend on the index, we use `for` loops. In Turing this is just the standard Julia code, but you need to remember to allocate an array to store the variables:

In [12]:
@model function demo(::Type{TV} = Vector{Float64}) where {TV}
    y = TV(undef, J)
    for j = 1:J
        Normal(θ[j], σ[j])
    end
end

demo (generic function with 3 methods)

By adding the type-parameter `TV` we allow Turing to internally replace `TV` with types that are compatible with whatever AD framework we're using at the time, e.g. if we're using ForwardDiff.jl (which is the case by default), `TV` is replaced by `Vector{ForwardDiff.Dual{Float64}}`. One could of course just do

In [13]:
@model function demo()
    y = Vector(undef, J)
    for j = 1:J
        Normal(θ[j], σ[j])
    end
end

demo (generic function with 3 methods)

but this would lead to type-unstable code and hence a loss in performance.

## Broadcasted statements, i.e. `.~`

In Turing you can also use the broadcasted version of `~` where the semantics are the same as the broadcasting behavior in Julia, e.g.

In [ ]:
x .~ Normal(0, 1)

and

In [ ]:
for i in eachindex(x)
    x[i] ~ Normal(0, 1)
end

are equivalent. Similarly, if `μ` is a vector, the following are equivalent

In [ ]:
x .~ Normal.(μ, 1)

and

In [ ]:
for i in eachindex(x)
    x[i] ~ Normal(μ[i], 1)
end

Note that one can also use the `@. expr...` macro from Julia which tells Turing to broadcast everything following `@.`, i.e. the following is equivalent to the above snippets:

In [ ]:
@. x ~ Normal(μ, 1)

This is something we'll use repeatedly throughout as, amongst other things, `.~` provides a convenient way of expressing a `Vector` of IID samples.

### Sampling with `.~`

One drawback with the `x .~ dist` is that it requires the left-hand side `x` to be defined, i.e. `x` either needs to be part of the arguments to the model and is thus considered an observation (unless it's `missing`), _or_ we need to allocate an `Array` to hold the result similar to what we did for for-loops earlier. Therefore we'll often see the following piece of code being used throughout

In [15]:
@model function demo(n; x=missing)
    # If `x` is `missing`, we consider it as a random variable to be sampled.
    if x === missing
        # Since trying to do `x[i] ~ ...` doesn't make sense when `x === missing`
        # we need to replace `x` with an `Array` that can hold the results.
        x = Vector(undef, n)
    end

    x .~ Normal()
end

demo (generic function with 3 methods)

As mentioned above in the section on for-loops, `Vector(undef, n)` will be type-unstable, and so the above should really be written

In [16]:
@model function demo(n, ::Type{T}; x=missing) where {T}
    # If `x` is `missing`, we consider it as a random variable to be sampled.
    if x === missing
        # Since trying to do `x[i] ~ ...` doesn't make sense when `x === missing`
        # we need to replace `x` with an `Array` that can hold the results.
        x = Vector{T}(undef, n)
    end

    x .~ Normal()
end

demo (generic function with 4 methods)

for optimal performance.

We can also make the statement a bit more compact:

In [17]:
@model function demo(n, ::Type{T}; x=missing) where {T}
    x = x === missing ? Vector{T}(undef, n) : x
    x .~ Normal()
end

demo (generic function with 4 methods)